## Problem 3
Authors: Aman Kumar (EE21B013) & Aayush Patel (EE21B003)

The below function converts the binary string to ASCII format

In [1]:
def binary_to_ascii(binary_string):
    chunks = [binary_string[i:i+8] for i in range(0, len(binary_string), 8)]
    ascii_string = ''.join(chr(int(chunk, 2)) for chunk in chunks)
    return ascii_string


The _read\_domain\_name_ function extracts the domain name (as s string) from the dns message

In [2]:
def read_domain_name(packet,st):
    domain =""
    while True:
        len_level=int(packet[st:st+8],2)

        if(len_level==0):
            break
        st=st+8
        label_name=packet[st:st+len_level*8]
        st=st+len_level*8
        temp=binary_to_ascii(label_name)
        domain+=temp
        domain+='.'
    st=st+8
    domain=domain[:len(domain)-1]
    return st,domain

In [3]:
# def read_com_domain_name(packet,domain,offset):
#     while True :
#         len_level=int(packet[offset:offset+8],2)

#         if(len_level==0):
#             break
#         if(packet[st]=='0' and packet[st+1]=='0'):
#             st=st+8
#             label_name=packet[st:st+len_level*8]
#             st=st+len_level*8
#             temp=binary_to_ascii(label_name)
#             # print(temp)
#             domain+=temp
#             domain+='.'

The function _parse\_dns\_packet_ parses the dns message to extract 
* domain name for query
* domain name and IP Address for response 

In [4]:
## fucntion for parsing the dns message
def parse_dns_packet(packet):
    header = packet[:96]
    # Extract fields from the DNS header
    transaction_id = int(header[0:16],2)
    # print('transaction_id',transaction_id)
    question_count=int (header[32:48],2)
    checker_flag=header[16] #check for query or response
    answer_count=int(header[48:64],2)
    if(question_count>1 or answer_count>1):
        raise Exception
    
    # # Check if it's a query or response packet
    if checker_flag == '0':  # Query packet
        # Extract questions from the DNS packet
        
        next_ind,domain=read_domain_name(packet,96)
        print('Packet Type : query')
        print('domain name : ',domain)
        next_ind+=32
        
    else :  # Response packet
        
        next_ind,domain=read_domain_name(packet,96)
        if(question_count): 
            next_ind+=32
            next_ind+=16
        ans_type=int(packet[next_ind:next_ind+16])
        next_ind+=16
        ans_class=int(packet[next_ind:next_ind+16])
        next_ind+=64
        if(ans_class!=1 and ans_type!=1):
            raise Exception
        # print(ans_class,ans_type)
        d=[]
        d1=int(packet[next_ind:next_ind+8],2)
        d.append(d1)
        next_ind+=8
        d2=int(packet[next_ind:next_ind+8],2)
        d.append(d2)
        next_ind+=8
        d3=int(packet[next_ind:next_ind+8],2)
        d.append(d3)
        next_ind+=8
        d4=int(packet[next_ind:next_ind+8],2)
        d.append(d4)
        ipv4=""
        for e in d:
            e1=str(e)
            ipv4+=e1
            ipv4+='.'
        ipv4=ipv4[:len(ipv4)-1]
        print('packet Type : Response')
        print('domain name : ',domain)
        print('Host IP address : ',ipv4)

#### Queries

* Add any new query in the queries list and run the next shell

In [5]:
queries = [
    '4c76010000010000000000010c74696d65736f66696e6469610a696e64696174696d657303636f6d00000100010000290200000000000000',
    '629f8180000100010000000103637365046969746d02616302696e0000010001c00c0001000100001f9600040a0608020000290200000000000000',
    '00c88b57ec40ec2e98e9046b08004500004b527e40004011802f0a2a52f10a1800c2e36400350037875137920100000100000000000106636c69656e740764726f70626f7803636f6d00000100010000290200000000000000',
    'ec2e98e9046b34e894fa3f5e08004500006b030400003e11f7c4c0a80001c0a80068003581c300577fa37baf818000010001000000010f7a2d7034322d696e7374616772616d046331307209696e7374616772616d03636f6d0000010001c00c000100010000000c00049df017ae0000290200000000000000',
    'a0208180000100010000000105666f6e74730a676f6f676c656170697303636f6d0000010001c00c00010001000000ae00048efab64a0000290200000000000000',
]

#### Run the following cell to get the output for the queries in the queries list.

In [6]:
## first converting into binary representation
for i, query in enumerate(queries, start=1):
    print(f"\nParsing Packet {i}:")
    P_query=""
    cnt=0
    for i in query:
        no=int(i,16)
        no=list(bin(no)[2:])
        while(len(no)<4):
            no.insert(0,'0')
        temp=''
        for i in no:
            temp+=i
        P_query+=temp
    try:
        parse_dns_packet(P_query)
    except:
        print("Invalid Input")


Parsing Packet 1:
Packet Type : query
domain name :  timesofindia.indiatimes.com

Parsing Packet 2:
packet Type : Response
domain name :  cse.iitm.ac.in
Host IP address :  10.6.8.2

Parsing Packet 3:
Invalid Input

Parsing Packet 4:
Invalid Input

Parsing Packet 5:
packet Type : Response
domain name :  fonts.googleapis.com
Host IP address :  142.250.182.74
